### Exp: 1 working with Snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check ,same type, same cam , timestamp +-2 x,y is near each other appear at least 2 timeframe on the same x,y on snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess

import countpassenger
import pandas as pd
import os.path as osp
import os

CURRENT_CAM = 'mbk-14-12'

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "mbk-tourist-vehicle-object-20240429.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(df_vehicle=df_vehicle)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM+'-vehicle')
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

#drop the one that lifetime is less than 4 seconds 
df_vehicle = df_vehicle.loc[
        (df_vehicle["lifetime"] > 4),
    ]

#create a timestamp_unix at the end of lifespan
df_vehicle["timestamp_unix_end"] = (df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"] *1.2))
# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())


/home/jeans/internship/parking-customer-count


In [2]:
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "mbk-tourist-raw-cross-object-20240429.csv")
)

df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
import pandas as pd
import numpy as np

def match_cross_to_vehicle(df_cross, df_vehicle, start_padding: int = 5, stop_padding: int = 20):
    
    df_vehicle_copy = df_vehicle.copy()
    df_vehicle_copy['cross_count'] = 0
    
    # Iterate over each row in df_cross
    for index, row in df_cross.iterrows():
        # Find the rows in df_vehicle where the timestamp is within the range
        mask = (df_vehicle_copy['timestamp_unix']-start_padding <= row['timestamp_unix']) & (df_vehicle_copy['timestamp_unix_end']+stop_padding >= row['timestamp_unix'])
        vehicle_rows = df_vehicle_copy[mask]
        if vehicle_rows.empty:
            # print('empty mask')
            continue
        
        # Calculate the distance from the current cross to each vehicle
        distances = np.sqrt((vehicle_rows['xmax'] - row['xmin'])**2 + (vehicle_rows['ymax'] - row['ymax'])**2)
        # print("dist", distances)
        # # Find the index of the nearest vehicle
        nearest_index = distances.idxmin(axis=0)
        
        # Increment the cross count and mark as used for the nearest vehicle
        df_vehicle_copy.loc[nearest_index, 'cross_count'] += 1
        # print(df_vehicle_copy.loc[nearest_index, 'cross_count'])
        # df_vehicle_copy.loc[nearest_row, 'used'] = True
    
    return df_vehicle_copy

In [4]:
df_vehicle_cross_count =  match_cross_to_vehicle(df_cross=df_cross, df_vehicle=df_vehicle)

In [5]:
df_vehicle_cross_count[[*conf.VEHICLE_INTEREST_SNAPSHOT, 'cross_count']]

,camera,timestamp_precise,xmin,xmax,ymin,ymax,lifetime,plate_number,similarized_plate_number,vehicle_type,vehicle_type_confidence,cross_count
1123,mbk-14-12-vehicle,2024-04-29 07:43:16.191,1219,1671,0,327,94.213000,1มก3812,1มก3812,truck,0.980706,2
1012,mbk-14-12-vehicle,2024-04-29 08:12:08.130,1397,2160,0,398,273.087000,341354,341354,bus,0.530526,2
949,mbk-14-12-vehicle,2024-04-29 08:40:56.629,1247,2083,0,370,578.126000,332185,332185,van,0.244582,7
855,mbk-14-12-vehicle,2024-04-29 09:25:56.229,950,1940,0,302,11.877000,ขท3660,ขท3660,van,0.344834,0
786,mbk-14-12-vehicle,2024-04-29 10:15:41.634,1411,2193,0,405,182.672000,341520,341520,bus,0.348732,22
733,mbk-14-12-vehicle,2024-04-29 10:32:03.931,1198,1643,4,302,216.303000,NaN,NaN,truck,0.910081,2
726,mbk-14-12-vehicle,2024-04-29 10:44:54.164,1102,1643,0,269,11.480000,ฐก4108,ฐก4108,truck,0.911475,3
687,mbk-14-12-vehicle,2024-04-29 10:54:43.886,1123,1690,0,302,63.627000,สก5839,ลก5839,bus,0.566742,3
688,mbk-14-12-vehicle,2024-04-29 10:56:11.226,1064,1704,2,358,10.517000,สก547,ลก547,truck,0.956156,0
632,mbk-14-12-vehicle,2024-04-29 11:34:57.602,1409,1905,0,388,4.797022,1มก9013,1มก9013,bus,0.348310,0
